In [1]:
import pandas as pd
import numpy as np
import os

from lib.io import load_pickle_data
from external_lib.gnss_transformer import get_gnss_data_path_from_phone,gnss_log_to_dataframes,load_gnss_raw_data_from_file
from tqdm import tqdm as default_tqdm
from tqdm.notebook import tqdm as notebook_tqdm

In [2]:
!ls /work/data/input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-1/Pixel4/

Pixel4_GnssLog.txt  Pixel4_derived.csv	ground_truth.csv  supplemental


In [3]:
train_df_path =  "/work/data/input/selfmade_dataset/baseline_with_derived_data_v2/train.pkl"
train_df = load_pickle_data(train_df_path)
train_df = train_df.rename(columns={"millisSinceGpsEpoch":"MillisSinceGpsEpoch"})

test_df_path =  "/work/data/input/selfmade_dataset/baseline_with_derived_data_v2/test.pkl"
test_df = load_pickle_data(train_df_path)
test_df = test_df.rename(columns={"millisSinceGpsEpoch":"MillisSinceGpsEpoch"})

In [ ]:
s = [];

for key, eac in notebook_tqdm(train_df.groupby("phone")):
    
    data_path = get_gnss_data_path_from_phone(key,istrain=True);            
    gnss_data_raw = load_gnss_raw_data_from_file(data_path,verbose = "none")
                            
    gnss_data_raw['MillisSinceGpsEpoch'] = np.floor( (gnss_data_raw['TimeNanos'] - gnss_data_raw['FullBiasNanos']) / 1000000.0).astype(int)
    
    raw_timestamps = gnss_data_raw['MillisSinceGpsEpoch'].unique()    
    derived_timestamps = eac['MillisSinceGpsEpoch'].unique()
    
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)    
    
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
        
    eac['MillisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], eac['MillisSinceGpsEpoch'])))
    
    s.append(eac);
    
    
    
    

        

  0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
from lib.io import save_data_as_csv_and_pkl

output = pd.concat(s).sort_index();
save_data_as_csv_and_pkl(output,"/work/data/input/selfmade_dataset/baseline_with_derived_data_v3/train.");